In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

# 미리 작성된 코드들은 수정할 수 없으며, 이외의 코드를 작성하시면 됩니다.

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

class Classifier(nn.Module):
    # 모델의 코드는 여기서 작성해주세요

    def __init__(self, drop_prob):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(32*32*3, 2048) # input size : 32*32*3 (RGB)
        self.linear2 = nn.Linear(2048, 1024)
        self.linear3 = nn.Linear(1024, 512)
        self.linear4 = nn.Linear(512, 10)       # output size : 10

        self.dropout = nn.Dropout(drop_prob)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        z1 = self.linear1(x)
        a1 = self.activation(z1)
        a1 = self.dropout(a1)

        z2 = self.linear2(a1)
        a2 = self.activation(z2)
        a2 = self.dropout(a2)
        
        z3 = self.linear3(a2)
        a3 = self.activation(z3)
        a3 = self.dropout(a3)
        
        output = self.linear4(a3)

        return output


if __name__ == "__main__":
    # 학습코드는 모두 여기서 작성해주세요

    train_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                                 train=True,
                                                 transform=transforms.ToTensor(),
                                                 download=True)
    test_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                                train=False,
                                                transform=transforms.ToTensor(),
                                                download=True)
    
    batch_size = 4096

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

    model = Classifier(0.5).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.95, 0.999)) 
    criterion = nn.CrossEntropyLoss()

    epochs = 350
    lmbd = 0.003

    total_batch_num = len(train_dataloader)

    for epoch in range(epochs):
      avg_cost = 0
      model.train()

      for b_x, b_y in train_dataloader:
        b_x = b_x.view(-1, 32*32*3).to(device)
        logits = model(b_x) # forward prop
        loss = criterion(logits, b_y.to(device)) # get cost

        reg = model.linear1.weight.pow(2.0).sum()
        reg += model.linear2.weight.pow(2.0).sum()
        reg += model.linear3.weight.pow(2.0).sum()

        loss += lmbd*reg/len(b_x)/len(b_x)/2.

        optimizer.zero_grad()
        loss.backward() # backward prop
        optimizer.step() # update parameters

        avg_cost += loss / total_batch_num # 모든 데이터셋에 대한 cost 값

      print('Epoch : {} / {}, cost : {}'.format(epoch+1, epochs, avg_cost))

    torch.save(model.state_dict(), 'model.pt')  # 학습된 모델을 저장하는 코드입니다.


Files already downloaded and verified
Files already downloaded and verified
Epoch : 1 / 350, cost : 2.349175214767456
Epoch : 2 / 350, cost : 2.1999223232269287
Epoch : 3 / 350, cost : 2.093247890472412
Epoch : 4 / 350, cost : 2.056260108947754
Epoch : 5 / 350, cost : 2.0174760818481445
Epoch : 6 / 350, cost : 1.9705772399902344
Epoch : 7 / 350, cost : 1.927689552307129
Epoch : 8 / 350, cost : 1.8961387872695923
Epoch : 9 / 350, cost : 1.867374300956726
Epoch : 10 / 350, cost : 1.841262698173523
Epoch : 11 / 350, cost : 1.818313479423523
Epoch : 12 / 350, cost : 1.7959706783294678
Epoch : 13 / 350, cost : 1.7774440050125122
Epoch : 14 / 350, cost : 1.7613338232040405
Epoch : 15 / 350, cost : 1.7407426834106445
Epoch : 16 / 350, cost : 1.7234339714050293
Epoch : 17 / 350, cost : 1.7046362161636353
Epoch : 18 / 350, cost : 1.6868185997009277
Epoch : 19 / 350, cost : 1.672501802444458
Epoch : 20 / 350, cost : 1.6529356241226196
Epoch : 21 / 350, cost : 1.6404573917388916
Epoch : 22 / 350,

In [9]:
# 학습된 모델의 성능을 평가하는 코드입니다.
# 아래의 코드로 평가를 진행할 예정이므로 아래의 코드가 정상 동작 해야하며, 제출전 모델의 성능을 확인하시면 됩니다.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

test_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=True)


test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10000)

classifier = Classifier(0.5).to(device)
classifier.load_state_dict(torch.load('model.pt'))
classifier.eval()


for data, label in test_dataloader:
    data = data.view(-1, 32 * 32 * 3).to(device)

    with torch.no_grad():
        logits = classifier(data)

        pred = torch.argmax(logits, dim=1)

        total = len(label)
        correct = torch.eq(pred, label.to(device)).sum()

        print("Accuracy on test set : {:.4f}%".format(100 * correct / total))

Files already downloaded and verified
Accuracy on test set : 58.6700%
